In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.utils import to_categorical


In [2]:
train_dir = 'D:/leaf-disease-detection/Train'  
test_dir = 'D:/leaf-disease-detection/Test'    


In [3]:
train_datagen = ImageDataGenerator(
    rescale=1./255, 
    rotation_range=40, 
    width_shift_range=0.2,
    height_shift_range=0.2, 
    shear_range=0.2, 
    zoom_range=0.2,
    horizontal_flip=True, 
    fill_mode='nearest'
)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir, 
    target_size=(224, 224), 
    batch_size=32, 
    class_mode='categorical'
)
test_generator = test_datagen.flow_from_directory(
    test_dir, 
    target_size=(224, 224), 
    batch_size=32, 
    class_mode='categorical'
)

Found 3299 images belonging to 8 classes.
Found 379 images belonging to 8 classes.


In [4]:
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)


In [5]:
for layer in base_model.layers:
    layer.trainable = False


In [6]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])


In [8]:
callbacks = [
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
    ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True)
]

In [9]:
try:
    model.fit(train_generator, epochs=20, validation_data=test_generator, callbacks=callbacks)
except tf.errors.OutOfRangeError as e:
    print(f"Error: {e}")


Epoch 1/20


C:\Users\shiva\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


104/104 ━━━━━━━━━━━━━━━━━━━━ 581s 6s/step - accuracy: 0.1735 - loss: 2.0686 - val_accuracy: 0.5805 - val_loss: 1.7662
Epoch 2/20
104/104 ━━━━━━━━━━━━━━━━━━━━ 558s 5s/step - accuracy: 0.3666 - loss: 1.7549 - val_accuracy: 0.6887 - val_loss: 1.5055
Epoch 3/20
104/104 ━━━━━━━━━━━━━━━━━━━━ 622s 6s/step - accuracy: 0.5658 - loss: 1.4956 - val_accuracy: 0.7652 - val_loss: 1.3058
Epoch 4/20
104/104 ━━━━━━━━━━━━━━━━━━━━ 540s 5s/step - accuracy: 0.6591 - loss: 1.2945 - val_accuracy: 0.7599 - val_loss: 1.1422
Epoch 5/20
104/104 ━━━━━━━━━━━━━━━━━━━━ 573s 6s/step - accuracy: 0.6942 - loss: 1.1521 - val_accuracy: 0.7704 - val_loss: 1.0173
Epoch 6/20
104/104 ━━━━━━━━━━━━━━━━━━━━ 536s 5s/step - accuracy: 0.7573 - loss: 1.0010 - val_accuracy: 0.8127 - val_loss: 0.9127
Epoch 7/20
104/104 ━━━━━━━━━━━━━━━━━━━━ 489s 5s/step - accuracy: 0.7870 - loss: 0.8993 - val_accuracy: 0.8047 - val_loss: 0.8270
Epoch 8/20
104/104 ━━━━━━━━━━━━━━━━━━━━ 491s 5s/step - accuracy: 0.8000 - loss: 0.8396 - val_accuracy: 0.817

In [10]:
model.load_weights('best_model.keras')

In [11]:
model.save('leaf_disease_model.keras')